In [1]:
!pip install tinydb 
!pip install pandas_ta
!pip install python-binance
# !pip install yfinance
!pip install mplfinance

  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218925 sha256=6b3766e4da897c0c35b939f0bf8114d4abba2a738d864e9159d8e360cb29d21a
  Stored in directory: c:\users\stb\appdata\local\pip\cache\wheels\54\4a\75\06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas-ta


In [9]:
from tinydb import TinyDB, Query
import os
import sys
import json
# from dotenv import load_dotenv
import asyncio
import threading
import datetime
from multiprocessing import Pool, cpu_count
import matplotlib.dates as mpl_dates
# Other modules
import pandas as pd
import pandas_ta as ta
from pandas.io import sql
import numpy as np
import matplotlib.pyplot as plt

# python-binance module
from binance.client import Client
from binance import BinanceSocketManager
from binance.enums import *

# BINANCE_KEY = os.getenv('BINANCE_KEY')
# BINANCE_SECRET = os.getenv('BINANCE_SECRET')
BINANCE_KEY = ''
BINANCE_SECRET = ''

# connect binance
client = Client(BINANCE_KEY, BINANCE_SECRET)

import time


In [10]:
def get_candlestick_one_token(symbol):
    df1 = pd.DataFrame(client.get_historical_klines(symbol,
                                                       client.KLINE_INTERVAL_1HOUR,
                                                       "7 day ago UTC"
                                                       ))
    df1.columns = ["Date", "Open", "High", "Low", "Close", "Volume",
                  "Close_time", "Qav", "Num_trades", "Taker_base_vol",
                  "Taker_quote_vol", "Is_best_match"]
    df_simple = df1[["Date", "Open", "High", "Low", "Close", "Volume"]]
    df_simple[["Open", "High", "Low", "Close", "Volume"]] = df_simple[["Open", "High", "Low", "Close", "Volume"]].apply(pd.to_numeric)
    df_simple['Date_readonly'] = df_simple['Date'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
    df_simple['Date'] = pd.to_datetime(df_simple['Date'])
    df_simple['Date'] = df_simple['Date'].apply(mpl_dates.date2num)
    df_simple['Date'] = df_simple['Date']*1000000
    
    return df_simple
    

In [11]:
def analyse_rsi(df):
    rsi = ta.rsi(df['Close'])
    last_rsi =  rsi.iloc[-1]
    return last_rsi

In [18]:
symbol = 'GALAUSDT'
df = get_candlestick_one_token(symbol)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
<ipython-input-10-8e14b8673da7>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simple['Date_readonly'] = df_simple['Date'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
<ipython-input-10-8e14b8673da7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [19]:
df

,Date,Open,High,Low,Close,Volume,Date_readonly
0,18885.375000,0.04511,0.04559,0.04430,0.04460,28901889.0,2021-09-15 16:00:00
1,18885.416667,0.04460,0.04482,0.04433,0.04446,19600202.0,2021-09-15 17:00:00
2,18885.458333,0.04446,0.04494,0.04391,0.04461,30440845.0,2021-09-15 18:00:00
3,18885.500000,0.04461,0.04706,0.04453,0.04559,62731362.0,2021-09-15 19:00:00
4,18885.541667,0.04563,0.04570,0.04436,0.04458,28510167.0,2021-09-15 20:00:00
...,...,...,...,...,...,...,...
163,18892.166667,0.10416,0.10542,0.10220,0.10231,58971692.0,2021-09-22 11:00:00
164,18892.208333,0.10240,0.10567,0.10226,0.10434,65688623.0,2021-09-22 12:00:00
165,18892.250000,0.10428,0.10830,0.10294,0.10750,84603092.0,2021-09-22 13:00:00
166,18892.291667,0.10745,0.11140,0.10600,0.10889,184878883.0,2021-09-22 14:00:00


In [20]:
analyse_rsi(df)

56.254616499493416